In [3]:
import numpy as np
import pandas as pd

from model_function_ss_speaker import model_fitting_sp
from model_function_ss_addressee import model_fitting_ad

import csv

In [67]:
    def book_read(book_name):
    
        book = pd.read_csv('predicted_books/baseline/'+book_name+'baselinepredicted.csv')
    
        # make addressees lists
    
        for i in book.index:
    
            if book.at[i,'dialogue'] != 'NO QUOTE':
    
                list_ad = []
    
                addressee = book.at[i,'addressee']
    
                for j in range(len(addressee)):
    
                    if addressee[j] == '\'':
    
                        word = ''
    
                        for k in range(j+1,len(addressee)):
    
                            if (addressee[k] != '\''):
    
                                word = word + addressee[k]
    
                            else:
                    
                                break
                    
            
                        list_ad.append(word)
    
    
                while ', ' in list_ad:
                        
                    list_ad.remove(', ')
    
                while ']' in list_ad:
                        
                    list_ad.remove(']')
                    


                # As the focus of ML based addressee attribution is to only predict single addressees, multi addressee entries should be np.nan
                
                if len(list_ad) == 1: # single addressee
                
                    book.at[i,'addressee'] = list_ad[0]

                else: # multi addresseees

                    book.at[i,'addressee'] = np.nan

        return book


In [35]:
# List of Novel Names to be checked

Novel_names = []

with open('pdnc_dataset/PDNC-Novel-index.csv','r') as file:

    reader = csv.reader(file)

    for i in reader:

        novel_name = ""

        for j in i[1]:
            if ((j != " ") and (j != "'") and (j != "-")):
                novel_name = novel_name + j

        
        Novel_names.append(novel_name)
        
    Novel_names.remove('NovelTitle')


In [55]:
# Final Accuracy table 

performance = [['Book_Name','Model','Speaker','Addressee','Full']]

results_ML = pd.read_csv('results_ML.csv')
results_BL = pd.read_csv('results_baseline.csv')

speaker_sum_ML = 0
addressee_sum_ML = 0
full_sum_ML = 0

speaker_sum_BL = 0
addressee_sum_BL = 0
full_sum_BL = 0


for i in Novel_names:

    # getting the results of each novel in the baseline algorithm

    ML_speaker_accuracy = list(results_ML.loc[(results_ML['book_name'] == i)]['speaker_ac'])[0]

    ML_addressee_accuracy = list(results_ML.loc[(results_ML['book_name'] == i)]['accuracy_ac'])[0]

    ML_full_accuracy = list(results_ML.loc[(results_ML['book_name'] == i)]['full_ac'])[0]

    BL_speaker_accuracy = list(results_BL[(results_BL['book_name'] == i)]['speaker_ac'])[0]

    BL_addressee_accuracy = list(results_BL[(results_BL['book_name'] == i)]['accuracy_ac'])[0]

    BL_full_accuracy = list(results_BL[(results_BL['book_name'] == i)]['full_ac'])[0]

    performance.append([i,'ML',ML_speaker_accuracy,ML_addressee_accuracy,ML_full_accuracy])

    performance.append([i,'BL',BL_speaker_accuracy,BL_addressee_accuracy,BL_full_accuracy])
    

    speaker_sum_ML = speaker_sum_ML + list(results_ML.loc[(results_ML['book_name'] == i)]['speaker_ac'])[0]

    addressee_sum_ML = addressee_sum_ML + list(results_ML.loc[(results_ML['book_name'] == i)]['accuracy_ac'])[0]

    full_sum_ML = full_sum_ML + list(results_ML.loc[(results_ML['book_name'] == i)]['full_ac'])[0]

    speaker_sum_BL = speaker_sum_BL + list(results_BL[(results_BL['book_name'] == i)]['speaker_ac'])[0]

    addressee_sum_BL = addressee_sum_BL + list(results_BL[(results_BL['book_name'] == i)]['accuracy_ac'])[0]

    full_sum_BL = full_sum_BL + list(results_BL[(results_BL['book_name'] == i)]['full_ac'])[0]

# averaging sums

speaker_sum_ML = speaker_sum_ML / 13

addressee_sum_ML = addressee_sum_ML / 13

full_sum_ML = full_sum_ML / 13

speaker_sum_BL = speaker_sum_BL / 13

addressee_sum_BL = addressee_sum_BL / 13

full_sum_BL = full_sum_BL / 13


# writing to csv

performance.append(['Average','Baseline',speaker_sum_BL,addressee_sum_BL,full_sum_BL])
performance.append(['Average','ML',speaker_sum_ML,addressee_sum_ML,full_sum_ML])


performance = pd.DataFrame(performance)

performance.to_csv('Tables/final_accuracy.csv')

In [47]:
# Measuring speaker attribution performance per syntactic category 


performance = [['Model','Explicit','Anaphoric','Implicit']] # initialising list 

explicit_sum_BL= 0
anaphoric_sum_BL = 0
implicit_sum_BL = 0

explicit_sum_ML = 0
anaphoric_sum_ML = 0
implicit_sum_ML = 0


for i in Novel_names:

    book_pred_BL = pd.read_csv('predicted_books/baseline/'+i+'baselinepredicted.csv') # predicted book 
    book_pred_ML = pd.read_csv('predicted_books/ML_Full/'+i+'MLFullpredicted.csv') # predicted book 

    book_real = pd.read_csv('annotated_books/'+i+'annotated.csv') # real book 

    # getting the filters (Baseline)

    book_pred_exp_BL = book_pred_BL[book_pred_BL['syntactic'] == 'Explicit']
    book_real_exp_BL = book_real[book_real['syntactic'] == 'Explicit']
    book_pred_ana_BL = book_pred_BL[book_pred_BL['syntactic'] == 'Anaphoric']
    book_real_ana_BL = book_real[book_real['syntactic'] == 'Anaphoric']
    book_pred_imp_BL = book_pred_BL[book_pred_BL['syntactic'] == 'Implicit']
    book_real_imp_BL = book_real[book_real['syntactic'] == 'Implicit']

    # getting the filters (ML)

    book_pred_exp_ML = book_pred_ML[book_pred_ML['syntactic'] == 'Explicit']
    book_real_exp_ML = book_real[book_real['syntactic'] == 'Explicit']
    book_pred_ana_ML = book_pred_ML[book_pred_ML['syntactic'] == 'Anaphoric']
    book_real_ana_ML = book_real[book_real['syntactic'] == 'Anaphoric']
    book_pred_imp_ML = book_pred_ML[book_pred_ML['syntactic'] == 'Implicit']
    book_real_imp_ML = book_real[book_real['syntactic'] == 'Implicit']


    # find accuracy for explicit

    count = 0

    for i in (book_pred_exp_BL.index): # same indices for pred and real

        if (book_pred_exp_BL.at[i,'speaker'] == book_real_exp_BL.at[i,'speaker']):

            count += 1

    explicit_sum_BL += (count / len(book_pred_exp_ML.index))

    count = 0

    for i in (book_pred_exp_ML.index): # same indices for pred and real

        if (book_pred_exp_ML.at[i,'speaker'] == book_real_exp_ML.at[i,'speaker']):

            count += 1

    explicit_sum_ML += (count / len(book_pred_exp_ML.index))


    # find accuracy for anaphoric

    count = 0

    for i in (book_pred_ana_BL.index): # same indices for pred and real

        if (book_pred_ana_BL.at[i,'speaker'] == book_real_ana_BL.at[i,'speaker']):

            count += 1

    anaphoric_sum_BL += (count / len(book_pred_ana_BL.index))

    count = 0

    for i in (book_pred_ana_ML.index): # same indices for pred and real

        if (book_pred_ana_ML.at[i,'speaker'] == book_real_ana_ML.at[i,'speaker']):

            count += 1

    anaphoric_sum_ML += (count / len(book_pred_ana_ML.index))


    # find accuracy for implicit

    count = 0

    for i in (book_pred_imp_BL.index): # same indices for pred and real

        if (book_pred_imp_BL.at[i,'speaker'] == book_real_imp_BL.at[i,'speaker']):

            count += 1
            
    implicit_sum_BL += (count / len(book_pred_imp_BL.index))

    count = 0

    for i in (book_pred_imp_ML.index): # same indices for pred and real

        if (book_pred_imp_ML.at[i,'speaker'] == book_real_imp_ML.at[i,'speaker']):

            count += 1
            
    implicit_sum_ML += (count / len(book_pred_imp_ML.index))


# averaging up the sums

explicit_sum_BL = explicit_sum_BL / 13
anaphoric_sum_BL = anaphoric_sum_BL / 13
implicit_sum_BL = implicit_sum_BL / 13

explicit_sum_ML = explicit_sum_ML / 13
anaphoric_sum_ML = anaphoric_sum_ML / 13
implicit_sum_ML = implicit_sum_ML / 13



performance.append(['baseline',explicit_sum_BL,anaphoric_sum_BL,implicit_sum_BL])
performance.append(['ML',explicit_sum_ML,anaphoric_sum_ML,implicit_sum_ML])


performance = pd.DataFrame(performance)

performance.to_csv('Tables/syntactic_performance.csv')

In [73]:
# A test to see if vocatives are being predicted as addressees (Table 8)

count_vocative = 0

count_vocative_found_BL = 0
count_vocative_found_ML = 0
count_vocative_found_ML_SA = 0

count_quotes = 0

for book_name in Novel_names:

    print(f'doing book {book_name}')

    character_info = pd.read_csv('pdnc_dataset/data/'+ book_name +'/character_info.csv')
    
    Characters = character_info['Main Name']
    Characters = list(Characters)
    
    Aliases = []
    
    
    for i in character_info.index:
        new_list = []
        for j in range(len(character_info.at[i,'Aliases'])):
            string = character_info.at[i,'Aliases']
            if string[j] == '\'':
                new = ''
                index = j+1
                while (string[index] != '\'' and index < (len(string)-1)):
                    new = new + string[index]
                    index += 1
                new_list.append(new)
    
        # removing ', ' and '' terms
    
        for j in new_list:
            if (j == ', ' or j == ''):
                new_list.remove(j)
        
        Aliases.append(new_list)


    # getting real book / all other predicted books

    book_real = pd.read_csv('annotated_books/' + book_name + 'annotated.csv')
    
    book_predicted_BL = book_read(book_name)
    book_predicted_ML = pd.read_csv('predicted_books/ML_Full/' + book_name + 'MLFullpredicted.csv')
    book_predicted_ML_SA = pd.read_csv('predicted_books/ML_SA/' + book_name + 'ML_SApredicted.csv')


    # searching for quotes with vocatives, also only ensuring single list  aswell

    correct_database = []

    for i in book_real.index:

        if type(book_real.at[i,'addressee']) == list:

            if len(book_real.at[i,'addressee']) == 1:

                new_row = book_real.loc[i]

                new_row.at['addressee'] = new_row.at['addressee'][0] # making single element a list

                correct_database.append(new_row)
                
            else: 
                
                new_row = book_real.loc[i]

                new_row.at['addressee'] = np.nan # not trying to predict this row

                correct_database.append(new_row)

        else:
    
            correct_database.append(book_real.loc[i])
    
    correct_database = pd.DataFrame(correct_database)


    book_predicted_dialogue = correct_database[correct_database['dialogue'] != 'NO QUOTE']


    correct_database = []

    for i in book_predicted_BL.index:

        if type(book_predicted_BL.at[i,'addressee']) == list:

            if len(book_predicted_BL.at[i,'addressee']) == 1:

                new_row = book_predicted_BL.loc[i]

                new_row.at['addressee'] = new_row.at['addressee'][0] # making single element a list

                correct_database.append(new_row)
                
            else: 
                
                new_row = book_predicted_BL.loc[i]

                new_row.at['addressee'] = np.nan # not trying to predict this row

                correct_database.append(new_row)

        else:
    
            correct_database.append(book_predicted_BL.loc[i])
    
    correct_database = pd.DataFrame(correct_database)


    book_predicted_BL = correct_database[correct_database['dialogue'] != 'NO QUOTE']


    for i in book_predicted_dialogue.index:

        vocative_present = False # placeholder to say if a vocative was found in the dialogue

        for char in range(len(Characters)):

            if (book_predicted_ML.at[i,'addressee'] is np.nan) == False:

                for alias in Aliases[char]:
    
                    if (', ' + alias) in book_predicted_dialogue.at[i,'dialogue']:
    
                        vocative_present = True
    
                        if Characters[char] == book_predicted_BL.at[i,'addressee']:
    
                            count_vocative_found_BL += 1
    
                        if Characters[char] == book_predicted_ML.at[i,'addressee']:
    
                            count_vocative_found_ML += 1
                            
                        if Characters[char] == book_predicted_ML_SA.at[i,'addressee']:
    
                            count_vocative_found_ML_SA += 1
    
                        
    
                    elif (alias + ', ') in book_predicted_dialogue.at[i,'dialogue']:
    
                        vocative_present = True
    
                        if Characters[char] == book_predicted_BL.at[i,'addressee']:
    
                            count_vocative_found_BL += 1
    
                        if Characters[char] == book_predicted_ML.at[i,'addressee']:
    
                            count_vocative_found_ML += 1
                            
                        if Characters[char] == book_predicted_ML_SA.at[i,'addressee']:
    
                            count_vocative_found_ML_SA += 1
    
        if vocative_present == True:

            count_vocative += 1

        count_quotes += 1


vocative_accuracy_BL = count_vocative_found_BL / count_vocative
vocative_accuracy_ML = count_vocative_found_ML / count_vocative
vocative_accuracy_ML_SA = count_vocative_found_ML_SA / count_vocative


vocative_prop = count_vocative / count_quotes


with open('Tables/vocative_performance.csv','w') as file:

    writer = csv.writer(file)

    writer.writerows([['model','vocative_accuracy'],['baseline',vocative_accuracy_BL],['ML',vocative_accuracy_ML],['ML_SA',vocative_accuracy_ML_SA]])

        


doing book AlicesAdventuresinWonderland
doing book AnneOfGreenGables
doing book Emma
doing book NorthangerAbbey
doing book Persuasion
doing book PrideandPrejudice
doing book SenseandSensibility
doing book TheAgeofInnocence
doing book TheAwakening
doing book TheInvisibleMan
doing book TheManWhoWasThursday
doing book ThePictureOfDorianGray
doing book TheSignoftheFour


In [51]:
# Looking at the top-ranking probabilities that were used to assign speakers and addressees,
# and their impact on results

for book_name in Novel_names:

    book_correct = pd.read_csv('Annotated_New/'+book_name + 'anotatednew.csv')
    
    book_ML = pd.read_csv('predicted_books/ML_Full/'+book_name+'MLFullpredicted.csv')
    
    prob_correct_sp_exp = 0
    prob_false_sp_exp = 0
    count_sp_exp = 0
    
    prob_correct_sp_ana = 0
    prob_false_sp_ana = 0
    count_sp_ana = 0
    
    prob_correct_sp_imp = 0
    prob_false_sp_imp = 0
    count_sp_imp = 0
    
    prob_correct_ad = 0
    prob_false_ad = 0
    count_ad = 0
    
    
    for i in book_correct.index:
    
        if book_correct.at[i,'dialogue'] != 'NO QUOTE':
    
            if book_correct.at[i,'speaker'] == book_ML.at[i,'speaker']:
    
                prob_correct_sp += book_ML.at[i,'probability_sp']
                count_sp += 1
    
            else:
    
                prob_false_sp += book_ML.at[i,'probability_sp']
                count_sp += 1
    
            if book_correct.at[i,'addressee'] == book_ML.at[i,'addressee']:
    
                prob_correct_ad += book_ML.at[i,'probability_ad']
                count_ad += 1
    
            else:
    
                if (book_correct.at[i,'addressee'] is np.nan) == False:
    
                    prob_false_ad += book_ML.at[i,'probability_ad']
                    count_ad += 1
    
            if book_correct.at[i,'syntactic'] == 'Explicit':
    
                if book_correct.at[i,'speaker'] == book_ML.at[i,'speaker']:
    
                    prob_correct_sp_exp += book_ML.at[i,'probability_sp']
                    count_sp_exp += 1
    
                else:
    
                    prob_false_sp_exp += book_ML.at[i,'probability_sp']
                    count_sp_exp += 1
                    
            if book_correct.at[i,'syntactic'] == 'Anaphoric':
    
                if book_correct.at[i,'speaker'] == book_ML.at[i,'speaker']:
    
                    prob_correct_sp_ana += book_ML.at[i,'probability_sp']
                    count_sp_ana += 1
    
                else:
    
                    prob_false_sp_ana += book_ML.at[i,'probability_sp']
                    count_sp_ana += 1
    
            if book_correct.at[i,'syntactic'] == 'Implicit':
    
                if book_correct.at[i,'speaker'] == book_ML.at[i,'speaker']:
    
                    prob_correct_sp_imp += book_ML.at[i,'probability_sp']
                    count_sp_imp += 1
    
                else:
    
                    prob_false_sp_imp += book_ML.at[i,'probability_sp']
                    count_sp_imp += 1
    
            
                


print(f' Average Correct Speaker Probability {prob_correct_sp/count_sp}')
print(f' Average False Speaker Probability {prob_false_sp/count_sp}')

print(f' Average Correct Addressee Probability {prob_correct_ad/count_ad}')
print(f' Average False Addressee Probability {prob_false_ad/count_ad}')

print(f' Average Correct Exp Probability {prob_correct_sp_exp/count_sp_exp}')
print(f' Average False Exp Probability {prob_false_sp_exp/count_sp_exp}')

print(f' Average Correct Ana Probability {prob_correct_sp_ana/count_sp_ana}')
print(f' Average False Ana Probability {prob_false_sp_ana/count_sp_ana}')

print(f' Average Correct Imp Probability {prob_correct_sp_imp/count_sp_imp}')
print(f' Average False Imp Probability {prob_false_sp_imp/count_sp_imp}')



 Average Correct Speaker Probability 0.45268111601385497
 Average False Speaker Probability 0.14349159715434032
 Average Correct Addressee Probability 0.3403095911190253
 Average False Addressee Probability 0.038299426264864905
 Average Correct Exp Probability 0.8847554262663075
 Average False Exp Probability 0.006575255892738235
 Average Correct Ana Probability 0.41846685447796916
 Average False Ana Probability 0.17792751518159217
 Average Correct Imp Probability 0.20242218734061004
 Average False Imp Probability 0.23893164239922854
